In [1]:
#|default_exp models.FCN

# FCN

This is an unofficial PyTorch implementation created by Ignacio Oguiza (oguiza@timeseriesAI.co)
based on:

* Wang, Z., Yan, W., & Oates, T. (2017, May). <span style="color:dodgerblue">**Time series classification
from scratch with deep neural networks: A strong baseline**</span>. 
In 2017 international joint conference on neural networks (IJCNN) (pp. 1578-1585). IEEE.

* Fawaz, H. I., Forestier, G., Weber, J., Idoumghar, L., & Muller, P. A. (2019). <span style="color:dodgerblue">**Deep learning
for time series classification: a review**</span>. Data Mining and Knowledge Discovery, 33(4),
917-963.

Official FCN TensorFlow implementation: https://github.com/hfawaz/dl-4-tsc/blob/master/classifiers/fcn.py.

Note: kernel filter size 8 has been replaced by 7 (since we believe it's a bug).

In [2]:
#|export
from tsai.imports import *
from tsai.models.layers import *

In [3]:
#|export
class FCN(Module):
    def __init__(self, c_in, c_out, layers=[128, 256, 128], kss=[7, 5, 3]):
        assert len(layers) == len(kss)
        self.convblock1 = ConvBlock(c_in, layers[0], kss[0])
        self.convblock2 = ConvBlock(layers[0], layers[1], kss[1])
        self.convblock3 = ConvBlock(layers[1], layers[2], kss[2])
        self.gap = GAP1d(1)
        self.fc = nn.Linear(layers[-1], c_out)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.gap(x)
        return self.fc(x)

In [43]:
import torch
import os
import numpy as np

# Carregar seus dados
folder_dementia = r"C:\Users\Lenovo\Desktop\IC\[99] Database Final (16_287)\cookie_d"
folder_control = r"C:\Users\Lenovo\Desktop\IC\[99] Database Final (16_287)\cookie_c/"

data_dementia = []
for npy_file in os.listdir(folder_dementia):
    if npy_file.endswith(".npy"):
        data = np.load(os.path.join(folder_dementia, npy_file))
        data_dementia.append(data)

data_control = []
for npy_file in os.listdir(folder_control):
    if npy_file.endswith(".npy"):
        data = np.load(os.path.join(folder_control, npy_file))
        data_control.append(data)

# Combine e processe seus dados
X = np.concatenate((data_dementia, data_control), axis=0)
y = np.concatenate((np.ones(len(data_dementia)), np.zeros(len(data_control))), axis=0)

# Seus parâmetros
bs = 16
nvars = 549  # Número de variáveis em seus dados
seq_len = 128  # Comprimento da sequência em seus dados
c_out = 2  # Número de classes (neste caso, positivo ou controle)

# Criar tensores de exemplo com seus dados
xb = torch.tensor(X, dtype=torch.float32)  # Converta para tensor
yb = torch.tensor(y, dtype=torch.int64)  # Converta para tensor de inteiros

# Verifique as dimensões dos tensores
print("xb shape:", xb.shape)
print("yb shape:", yb.shape)


print(xb,yb)
# Criar modelo FCN
model = FCN(nvars, c_out)

# Passar dados de exemplo pelo modelo
preds = model(xb)

# Calcular a acurácia
acc = accuracy(preds, yb)

print(f"Acurácia do modelo: {acc.item():.4f}")

xb shape: torch.Size([549, 287])
yb shape: torch.Size([549])
tensor([[-3.0518e-05,  3.0518e-05, -3.0518e-05,  ..., -9.1553e-05,
          0.0000e+00,  0.0000e+00],
        [ 2.1362e-04, -2.4414e-04,  1.2207e-04,  ...,  9.1553e-05,
         -9.1553e-05,  0.0000e+00],
        [-1.2207e-04,  0.0000e+00,  9.1553e-05,  ...,  6.1035e-05,
         -3.0518e-05, -3.0518e-05],
        ...,
        [-3.9673e-04, -3.9673e-04,  3.0518e-04,  ...,  0.0000e+00,
         -6.1035e-05,  2.4414e-04],
        [-3.0518e-05,  1.5259e-04, -9.1553e-05,  ..., -1.8311e-04,
          0.0000e+00, -1.2207e-04],
        [-3.0518e-05, -3.0518e-05, -3.0518e-05,  ..., -6.1035e-05,
         -3.0518e-05, -3.0518e-05]]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

RuntimeError: running_mean should contain 287 elements not 128

In [8]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

c:\Users\Lenovo\Desktop\IC\tsai\nbs\038_models.FCN.ipynb saved at 2023-08-18 09:44:25
Correct notebook to script conversion! 😃
Friday 18/08/23 09:44:28 Hora oficial do Brasil
